## Generate feature extracted data based on window size, overlap and class numbers.
1. Extract features with three class labels

In [1]:
import sys
sys.path.append('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch')
import os
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from essentials import three_class_labels, normalize
from feature_set_4 import GenerateFeatures 
import copy

In [2]:
with open('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/data/df_dict_imu.pkl', 'rb') as f:
    imu_dict = pickle.load(f)
with open('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/data/df_minze_dict.pkl', 'rb') as f:
    ground_truth_dict = pickle.load(f)
with open('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/data/df_dict_urineestimate_method1.pkl', 'rb') as f:
    urine_estimate_dict = pickle.load(f)

Remove the data with the almost the entire data being void.
1. subj_9_void4
2. subj_11_void2

In [3]:
del imu_dict['subj_9_void4']
del imu_dict['subj_11_void2']

Extract windowed features from the 3 axis of the accelerometer and the acceleration magnitude.
1. mean
2. rms
3. std
4. range
5. variance
6. min
7. max
8. time energy
9. spectral energy
10. permutation entropy
11. spectral entropy
12. spectral_centroid
13. spectral_spread
14. spectral_flatness
15. peak_frequency_ratio
16. hjorth_frequency
17. spectral_skewness
18. spectral_kurtosis
19. snr


## Three class data

## Data 6
classes: pre-void, void, non-void

fs: original

overlap: 0.0

sliding window: 1s

filename: three_class_raw_1s_no.csv

In [4]:
labelled_imu_dict = {}
dict = copy.deepcopy(imu_dict)
for i_void_instance, void_instance in tqdm(enumerate(dict.keys()), desc="Adding labels to IMU data"):
    acc = dict[void_instance]
    gt = ground_truth_dict[void_instance]
    
    # normalize data
    df = normalize(acc)
    
    # add the labels
    labelled_df = three_class_labels(df, gt)
    
    labelled_imu_dict[void_instance] = labelled_df

Adding labels to IMU data: 41it [00:00, 318.75it/s]


In [5]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data]    
    fs = 1 / df['time'].diff().median()
    analyzer = GenerateFeatures(fs, window_duration=1.0, overlap=0.0)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing signals: 100%|██████████| 5/5 [00:00<00:00, 27.97it/s]


In [6]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/feature_set_4/three_class_raw_1s_no.csv', index=False)
final_features

,center_time,acc_x_mean,acc_x_std,acc_x_energy,acc_x_jerk,acc_x_autocorr_peak,acc_x_autocorr_lag,acc_x_sample_entropy,acc_x_dominant_frequency,acc_x_spectral_centroid,...,gyr_mag_high_band_power,acc_xy_corr,acc_xz_corr,acc_yz_corr,acc_gyr_corr,tilt_var,acc_magnitude_mean,gyr_magnitude_mean,label,experiment_id
0,0.499593,0.938842,1.134729,2.169033,27.488924,0.912208,0.016935,0.955511,5.004075,5.012356,...,0.298711,0.037201,-0.056033,0.539271,0.249948,0.133549,1.747646,2.784556,pre-void,1
1,1.498779,1.115575,0.848165,1.963893,21.061566,0.935280,0.016935,0.771109,2.001630,3.190464,...,0.081451,0.237416,-0.180868,0.769078,0.217836,0.133930,1.545172,2.320707,pre-void,1
2,2.497964,0.804784,0.245606,0.707999,9.214712,0.964494,0.016935,1.791759,2.001630,6.507641,...,0.018886,-0.237806,0.203240,0.323739,-0.062160,0.021656,1.154535,0.494648,pre-void,1
3,3.497150,0.720945,0.292908,0.605557,9.463613,0.951214,0.016935,1.236763,6.004890,7.154587,...,0.024478,-0.135918,-0.089793,0.590726,0.116232,0.042694,1.083606,0.871746,pre-void,1
4,4.496336,0.588285,0.231525,0.399683,10.304930,0.936266,0.016935,1.845827,6.004890,8.345464,...,0.029350,0.087680,0.026996,0.639205,-0.156176,0.031684,1.923561,0.862959,pre-void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2407,36.229895,-0.554569,0.075345,0.313224,2.936831,0.976682,0.016543,1.203973,1.007455,11.477420,...,0.018306,-0.034151,0.064154,0.861385,-0.239621,0.006561,0.702208,0.554774,post-void,41
2408,37.222495,-0.532731,0.070325,0.288748,2.906265,0.977882,0.016543,1.435085,8.059642,10.793874,...,0.028689,0.333514,0.390338,0.938355,0.111316,0.006603,0.842645,0.830232,post-void,41
2409,38.215095,-0.264320,0.091840,0.078299,3.288201,0.952041,0.016543,2.639057,1.007455,4.632146,...,0.011328,-0.306270,-0.311402,0.835798,0.427657,0.007968,0.642942,0.810069,post-void,41
2410,39.207694,-0.251440,0.084821,0.070417,3.106321,0.941264,0.016543,1.137079,8.059642,10.927177,...,0.022779,-0.067292,-0.008764,0.868210,-0.285454,0.009143,0.662928,0.603053,post-void,41


## Data 6b
classes: pre-void, void, non-void

fs: original

overlap: 0.5

sliding window: 1s

filename: three_class_raw_1s_yo_0.5.csv

In [7]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data]    
    fs = 1 / df['time'].diff().median()
    analyzer = GenerateFeatures(fs, window_duration=1.0, overlap=0.5)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing signals: 100%|██████████| 5/5 [00:00<00:00, 14.09it/s]


In [8]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/feature_set_4/three_class_raw_1s_0.5.csv', index=False)

## Data 6c
classes: pre-void, void, non-void

fs: original

overlap: 0.8

sliding window: 1s

filename: three_class_raw_1s_yo_0.8.csv

In [9]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data]    
    fs = 1 / df['time'].diff().median()
    analyzer = GenerateFeatures(fs, window_duration=1.0, overlap=0.8)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing signals: 100%|██████████| 5/5 [00:00<00:00,  5.78it/s]


In [10]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/feature_set_4/three_class_raw_1s_0.8.csv', index=False)
final_features

,center_time,acc_x_mean,acc_x_std,acc_x_energy,acc_x_jerk,acc_x_autocorr_peak,acc_x_autocorr_lag,acc_x_sample_entropy,acc_x_dominant_frequency,acc_x_spectral_centroid,...,gyr_mag_high_band_power,acc_xy_corr,acc_xz_corr,acc_yz_corr,acc_gyr_corr,tilt_var,acc_magnitude_mean,gyr_magnitude_mean,label,experiment_id
0,0.499593,0.938842,1.134729,2.169033,27.488924,0.912208,0.016935,0.955511,5.004075,5.012356,...,0.298711,0.037201,-0.056033,0.539271,0.249948,0.133549,1.747646,2.784556,pre-void,1
1,0.702817,0.983020,1.152208,2.293911,30.398463,0.884909,0.016935,1.170071,5.004075,4.658724,...,0.179452,0.293840,0.101216,0.667022,0.371200,0.144950,1.737057,2.965490,pre-void,1
2,0.906041,1.225253,1.212918,2.972414,32.325434,0.886697,0.016935,1.021651,5.004075,4.823689,...,0.329331,0.211464,0.019451,0.741511,0.232239,0.140417,1.936316,3.099060,pre-void,1
3,1.109266,1.268793,1.182149,3.007312,30.433342,0.922150,0.016935,1.170071,6.004890,5.329942,...,0.317698,0.244637,-0.020904,0.709575,0.094679,0.153499,1.900757,3.043200,pre-void,1
4,1.312490,1.142122,1.200226,2.744983,27.668439,0.895832,0.016935,0.659246,6.004890,4.437748,...,0.266304,0.346269,-0.040062,0.719099,0.242046,0.169619,1.736450,2.873066,pre-void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11842,39.803254,-0.303074,0.103778,0.102624,2.915309,0.964470,0.016543,1.665008,1.007455,3.731236,...,0.027151,0.030177,0.084670,0.874391,-0.241898,0.008692,0.693343,0.750871,post-void,41
11843,40.001774,-0.303429,0.103177,0.102715,2.877756,0.968263,0.016543,1.609438,1.007455,3.536665,...,0.033527,0.045772,0.106295,0.875288,-0.212994,0.009060,0.701621,0.809929,post-void,41
11844,40.200294,-0.335019,0.109240,0.124171,2.837198,0.954422,0.016543,1.326871,2.014911,5.222870,...,0.034294,0.126612,0.078832,0.879146,-0.143812,0.008459,0.711812,0.884724,post-void,41
11845,40.398814,-0.386488,0.100174,0.159408,3.312941,0.961475,0.016543,1.321756,1.007455,5.143510,...,0.023032,0.252465,0.206606,0.878739,-0.082563,0.006644,0.767208,0.855583,post-void,41


## Data 7
classes: pre-void, void, non-void

fs: original

overlap: 0.0

sliding window: 2s

filename: three_class_raw_2s_no.csv

In [11]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data]    
    fs = 1 / df['time'].diff().median()
    analyzer = GenerateFeatures(fs, window_duration=2.0, overlap=0.0)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing signals: 100%|██████████| 5/5 [00:00<00:00, 51.44it/s]


In [12]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/feature_set_4/three_class_raw_2s_no.csv', index=False)
final_features

,center_time,acc_x_mean,acc_x_std,acc_x_energy,acc_x_jerk,acc_x_autocorr_peak,acc_x_autocorr_lag,acc_x_sample_entropy,acc_x_dominant_frequency,acc_x_spectral_centroid,...,gyr_mag_high_band_power,acc_xy_corr,acc_xz_corr,acc_yz_corr,acc_gyr_corr,tilt_var,acc_magnitude_mean,gyr_magnitude_mean,label,experiment_id
0,0.999186,1.027208,1.005637,2.066463,24.702545,0.922460,0.016935,0.758286,5.504482,4.866430,...,0.433022,0.113884,-0.069472,0.617961,0.241386,0.161761,1.646409,2.552632,pre-void,1
1,2.997557,0.762864,0.273525,0.656778,9.516582,0.962838,0.016935,1.414307,7.005705,7.133455,...,0.031394,-0.167799,0.012058,0.459326,0.009044,0.032951,1.119071,0.683197,pre-void,1
2,4.995929,0.381709,0.479763,0.375875,12.122137,0.833639,0.016935,1.396447,0.500407,4.853211,...,0.051982,-0.058582,-0.204570,0.700326,-0.094720,0.080964,1.800085,1.421490,pre-void,1
3,6.994300,0.842092,0.846989,1.426509,22.014882,0.888897,0.016935,1.076139,6.004890,4.992569,...,0.251759,-0.065512,0.036206,0.701680,0.160792,0.158051,1.529054,1.713838,pre-void,1
4,8.992671,0.565058,0.443517,0.515997,14.184323,0.899162,0.016935,1.402126,7.005705,6.824539,...,0.154371,-0.068425,0.226174,0.651821,0.276912,0.377185,1.383269,1.847173,pre-void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,31.027017,0.185532,0.356244,0.161332,4.772224,0.930323,0.016543,0.625895,0.499565,2.141629,...,0.113644,-0.348660,-0.511067,0.916241,-0.044309,0.159638,0.497629,2.392817,post-void,41
1185,33.028760,0.818064,0.115337,0.682532,3.628671,0.989482,0.016543,1.475907,0.999129,6.623307,...,0.052257,0.208354,0.219626,0.886649,-0.180890,0.021958,0.866123,1.005636,post-void,41
1186,35.030503,-0.145536,0.414586,0.193062,3.534657,0.962356,0.016543,0.462191,0.499565,1.249254,...,0.068363,-0.461865,-0.650086,0.828341,-0.642996,0.070416,0.526177,1.360635,post-void,41
1187,37.032246,-0.516767,0.106257,0.278338,2.970260,0.982923,0.016543,1.072637,0.499565,9.365089,...,0.067176,0.358593,0.486637,0.919841,0.102501,0.015038,0.792564,0.798071,post-void,41


## Data 7b
classes: pre-void, void, non-void

fs: original

overlap: 0.5

sliding window: 2s

filename: three_class_raw_2s_yo_0.5.csv

In [13]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data]    
    fs = 1 / df['time'].diff().median()
    analyzer = GenerateFeatures(fs, window_duration=2.0, overlap=0.5)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing signals: 100%|██████████| 5/5 [00:00<00:00, 24.99it/s]


In [14]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/feature_set_4/three_class_raw_2s_0.5.csv', index=False)
final_features

,center_time,acc_x_mean,acc_x_std,acc_x_energy,acc_x_jerk,acc_x_autocorr_peak,acc_x_autocorr_lag,acc_x_sample_entropy,acc_x_dominant_frequency,acc_x_spectral_centroid,...,gyr_mag_high_band_power,acc_xy_corr,acc_xz_corr,acc_yz_corr,acc_gyr_corr,tilt_var,acc_magnitude_mean,gyr_magnitude_mean,label,experiment_id
0,0.999186,1.027208,1.005637,2.066463,24.702545,0.922460,0.016935,0.758286,5.504482,4.866430,...,0.433022,0.113884,-0.069472,0.617961,0.241386,0.161761,1.646409,2.552632,pre-void,1
1,1.998371,0.960179,0.643429,1.335946,15.046835,0.946091,0.016935,0.846495,2.502037,4.024960,...,0.026769,0.134115,-0.190618,0.687476,0.326583,0.081498,1.349853,1.407678,pre-void,1
2,2.997557,0.762864,0.273525,0.656778,9.516582,0.962838,0.016935,1.414307,7.005705,7.133455,...,0.031394,-0.167799,0.012058,0.459326,0.009044,0.032951,1.119071,0.683197,pre-void,1
3,3.996743,0.654615,0.272212,0.502620,9.864408,0.945347,0.016935,1.296888,5.504482,6.614445,...,0.121658,-0.149286,-0.216850,0.734273,-0.017657,0.067167,1.503584,0.867352,pre-void,1
4,4.995929,0.381709,0.479763,0.375875,12.122137,0.833639,0.016935,1.396447,0.500407,4.853211,...,0.051982,-0.058582,-0.204570,0.700326,-0.094720,0.080964,1.800085,1.421490,pre-void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2361,35.741867,-0.476250,0.133958,0.244759,3.323721,0.982164,0.016543,1.408767,0.499565,7.531672,...,0.044109,-0.170184,-0.224131,0.876744,-0.638962,0.018042,0.626773,0.833723,post-void,41
2362,36.734466,-0.541521,0.077004,0.299175,2.929258,0.986050,0.016543,1.328536,0.499565,9.639247,...,0.051710,0.181962,0.265529,0.936136,0.124743,0.010973,0.770362,0.706243,post-void,41
2363,37.727066,-0.397643,0.156814,0.182711,3.105101,0.976195,0.016543,0.886829,0.499565,3.303041,...,0.110439,-0.152894,-0.178006,0.893149,0.214224,0.013437,0.741940,0.816687,post-void,41
2364,38.719666,-0.257434,0.088402,0.074087,3.196974,0.953721,0.016543,1.629241,0.499565,7.561225,...,0.050135,-0.188200,-0.169465,0.846853,0.186413,0.008558,0.653247,0.711023,post-void,41


## Data 7c
classes: pre-void, void, non-void

fs: original

overlap: 0.8

sliding window: 2s

filename: three_class_raw_2s_yo_0.8.csv

In [15]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data]    
    fs = 1 / df['time'].diff().median()
    analyzer = GenerateFeatures(fs, window_duration=2.0, overlap=0.8)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing signals: 100%|██████████| 5/5 [00:00<00:00, 10.35it/s]


In [16]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/feature_set_4/three_class_raw_2s_0.8.csv', index=False)
final_features

,center_time,acc_x_mean,acc_x_std,acc_x_energy,acc_x_jerk,acc_x_autocorr_peak,acc_x_autocorr_lag,acc_x_sample_entropy,acc_x_dominant_frequency,acc_x_spectral_centroid,...,gyr_mag_high_band_power,acc_xy_corr,acc_xz_corr,acc_yz_corr,acc_gyr_corr,tilt_var,acc_magnitude_mean,gyr_magnitude_mean,label,experiment_id
0,0.999186,1.027208,1.005637,2.066463,24.702545,0.922460,0.016935,0.758286,5.504482,4.866430,...,0.433022,0.113884,-0.069472,0.617961,0.241386,0.161761,1.646409,2.552632,pre-void,1
1,1.405634,1.034451,0.929655,1.934348,21.458067,0.921858,0.016935,0.712692,5.504482,4.274918,...,0.282971,0.167962,-0.102216,0.644390,0.395383,0.161737,1.535310,2.119416,pre-void,1
2,1.812083,0.978279,0.880376,1.732091,18.060970,0.912551,0.016935,0.633043,2.502037,3.496527,...,0.059930,0.250826,-0.092330,0.657934,0.395574,0.114310,1.451056,1.686827,pre-void,1
3,2.218531,0.898469,0.559360,1.120130,12.050941,0.951162,0.016935,0.943690,6.505297,5.905786,...,0.025770,-0.165077,-0.474689,0.603798,0.185676,0.074821,1.252705,1.165425,pre-void,1
4,2.624979,0.776798,0.255682,0.668788,8.368345,0.970977,0.016935,1.342674,7.005705,6.944119,...,0.027550,-0.185088,0.071125,0.440630,-0.019086,0.030820,1.090282,0.542118,pre-void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5822,38.322626,-0.303416,0.131681,0.109401,3.107525,0.962683,0.016543,1.060391,0.499565,3.954413,...,0.051882,-0.346402,-0.379857,0.879745,0.341358,0.010471,0.687180,0.814189,post-void,41
5823,38.719666,-0.257434,0.088402,0.074087,3.196974,0.953721,0.016543,1.629241,0.499565,7.561225,...,0.050135,-0.188200,-0.169465,0.846853,0.186413,0.008558,0.653247,0.711023,post-void,41
5824,39.116706,-0.250697,0.092173,0.071345,3.111349,0.947917,0.016543,1.580450,8.492598,9.503543,...,0.044763,-0.131523,-0.081729,0.858617,-0.051659,0.009450,0.636059,0.649435,post-void,41
5825,39.513746,-0.275963,0.094892,0.085160,2.936977,0.963100,0.016543,1.329724,1.498694,7.063329,...,0.045301,-0.032944,0.025586,0.865877,-0.141871,0.009938,0.667276,0.694077,post-void,41


## Data 8
classes: pre-void, void, non-void

fs: original

overlap: 0.0

sliding window: 3s

filename: three_class_raw_3s_no.csv

In [17]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data]    
    fs = 1 / df['time'].diff().median()
    analyzer = GenerateFeatures(fs, window_duration=3.0, overlap=0.0)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing signals: 100%|██████████| 5/5 [00:00<00:00, 69.46it/s]


In [18]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/feature_set_4/three_class_raw_3s_no.csv', index=False)
final_features

,center_time,acc_x_mean,acc_x_std,acc_x_energy,acc_x_jerk,acc_x_autocorr_peak,acc_x_autocorr_lag,acc_x_sample_entropy,acc_x_dominant_frequency,acc_x_spectral_centroid,...,gyr_mag_high_band_power,acc_xy_corr,acc_xz_corr,acc_yz_corr,acc_gyr_corr,tilt_var,acc_magnitude_mean,gyr_magnitude_mean,label,experiment_id
0,1.498779,0.953067,0.839825,1.613642,19.483560,0.930302,0.016935,0.732464,5.535758,4.757182,...,0.406946,0.076387,-0.096975,0.600847,0.349407,0.133637,1.482451,1.866637,pre-void,1
1,4.496336,0.494788,0.455654,0.452436,11.220113,0.886151,0.016935,1.195315,5.535758,6.768689,...,0.114098,-0.175302,-0.361143,0.704438,0.080459,0.094265,1.561259,1.238242,pre-void,1
2,7.493893,0.758589,0.711104,1.081126,17.810583,0.896869,0.016935,0.965959,5.997071,4.963067,...,0.247453,-0.084767,-0.085036,0.646938,0.133341,0.308302,1.479014,1.671782,pre-void,1
3,10.491450,0.635391,0.492101,0.645885,14.363947,0.920285,0.016935,1.425515,0.922626,4.259788,...,0.183335,0.026091,0.270384,0.583386,0.296998,0.263869,1.314198,1.602418,pre-void,1
4,13.489007,0.386718,0.475147,0.375315,12.953937,0.872672,0.016935,1.260717,2.767879,5.953986,...,0.277723,0.047597,0.282095,0.592904,0.424169,0.160867,1.144205,1.186102,pre-void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
784,25.451915,0.660903,0.121040,0.451443,3.705453,0.984602,0.016543,1.355609,0.472245,6.071340,...,0.056202,0.271539,0.235192,0.955698,0.403166,0.014484,1.174643,0.992954,void,41
785,28.446258,0.764856,0.154842,0.608981,3.921325,0.986896,0.016543,1.161839,0.472245,6.374299,...,0.087263,0.208736,0.215467,0.906451,0.034508,0.024827,0.942139,0.986867,post-void,41
786,31.440601,0.395153,0.419851,0.332421,4.383106,0.972319,0.016543,0.517282,0.472245,1.955933,...,0.123168,0.176062,0.063522,0.946644,-0.421256,0.149412,0.622435,1.938271,post-void,41
787,34.434943,0.220365,0.568465,0.371713,3.683369,0.983706,0.016543,0.329282,0.472245,1.669392,...,0.108771,-0.169456,-0.411244,0.821799,-0.498936,0.068885,0.646908,1.281264,post-void,41


## Data 8b
classes: pre-void, void, non-void

fs: original

overlap: 0.5

sliding window: 3s

filename: three_class_raw_3s_yo_0.5.csv

In [19]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data]    
    fs = 1 / df['time'].diff().median()
    analyzer = GenerateFeatures(fs, window_duration=3.0, overlap=0.5)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing signals: 100%|██████████| 5/5 [00:00<00:00, 35.29it/s]


In [20]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/feature_set_4/three_class_raw_3s_0.5.csv', index=False)
final_features

,center_time,acc_x_mean,acc_x_std,acc_x_energy,acc_x_jerk,acc_x_autocorr_peak,acc_x_autocorr_lag,acc_x_sample_entropy,acc_x_dominant_frequency,acc_x_spectral_centroid,...,gyr_mag_high_band_power,acc_xy_corr,acc_xz_corr,acc_yz_corr,acc_gyr_corr,tilt_var,acc_magnitude_mean,gyr_magnitude_mean,label,experiment_id
0,1.498779,0.953067,0.839825,1.613642,19.483560,0.930302,0.016935,0.732464,5.535758,4.757182,...,0.406946,0.076387,-0.096975,0.600847,0.349407,0.133637,1.482451,1.866637,pre-void,1
1,2.989089,0.731636,0.286953,0.617634,9.518142,0.959767,0.016935,1.411828,6.919697,6.960362,...,0.029453,-0.108568,-0.084068,0.552839,-0.018445,0.058893,1.185820,0.833215,pre-void,1
2,4.479400,0.483426,0.432560,0.420808,11.130985,0.905194,0.016935,1.201422,5.535758,6.759804,...,0.114253,-0.105224,-0.295491,0.677153,0.052231,0.087352,1.548907,1.222331,pre-void,1
3,5.969711,0.548849,0.809403,0.956368,19.209790,0.896639,0.016935,1.176905,0.922626,3.003914,...,0.129794,-0.171039,-0.257879,0.679567,-0.003780,0.418064,1.697595,1.640069,pre-void,1
4,7.460022,0.776208,0.723594,1.126086,18.111692,0.915078,0.016935,0.952506,5.997071,4.965532,...,0.254077,-0.108291,-0.105016,0.655747,0.167473,0.299263,1.497533,1.700828,pre-void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555,32.764067,0.617709,0.389886,0.533575,4.084924,0.988921,0.016543,0.517922,0.472245,1.375220,...,0.044278,0.353457,0.282356,0.892929,-0.726541,0.048416,0.746150,1.589861,post-void,41
1556,34.252967,0.295937,0.552217,0.392523,3.639527,0.985763,0.016543,0.354412,0.472245,1.929761,...,0.106457,-0.105673,-0.323662,0.830298,-0.544078,0.067406,0.662218,1.276728,post-void,41
1557,35.741867,-0.366768,0.314805,0.233621,3.311919,0.980933,0.016543,0.544643,0.472245,1.859945,...,0.064799,-0.560601,-0.647829,0.897463,-0.716729,0.046063,0.615118,1.043572,post-void,41
1558,37.230766,-0.449663,0.154201,0.225975,3.050020,0.983733,0.016543,0.885965,0.472245,8.890633,...,0.058613,0.129760,0.162830,0.912583,0.199245,0.020227,0.728769,0.729865,post-void,41


## Data 8c
classes: pre-void, void, non-void

fs: original

overlap: 0.8

sliding window: 3s

filename: three_class_raw_3s_yo_0.8.csv

In [21]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data]    
    fs = 1 / df['time'].diff().median()
    analyzer = GenerateFeatures(fs, window_duration=3.0, overlap=0.8)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing signals: 100%|██████████| 5/5 [00:00<00:00, 14.22it/s]


In [22]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/feature_set_4/three_class_raw_3s_0.8.csv', index=False)
final_features

,center_time,acc_x_mean,acc_x_std,acc_x_energy,acc_x_jerk,acc_x_autocorr_peak,acc_x_autocorr_lag,acc_x_sample_entropy,acc_x_dominant_frequency,acc_x_spectral_centroid,...,gyr_mag_high_band_power,acc_xy_corr,acc_xz_corr,acc_yz_corr,acc_gyr_corr,tilt_var,acc_magnitude_mean,gyr_magnitude_mean,label,experiment_id
0,1.498779,0.953067,0.839825,1.613642,19.483560,0.930302,0.016935,0.732464,5.535758,4.757182,...,0.406946,0.076387,-0.096975,0.600847,0.349407,0.133637,1.482451,1.866637,pre-void,1
1,2.091516,0.937189,0.752365,1.444375,16.064913,0.939576,0.016935,0.701166,5.997071,3.735016,...,0.124434,0.107435,-0.173207,0.632793,0.439577,0.124356,1.360390,1.394412,pre-void,1
2,2.684253,0.836695,0.516392,0.966720,11.176730,0.951540,0.016935,0.922687,6.919697,5.660981,...,0.027751,-0.080806,-0.370532,0.613729,0.224472,0.073114,1.227840,1.099562,pre-void,1
3,3.276990,0.724808,0.273863,0.600348,9.715540,0.955237,0.016935,1.440708,6.919697,7.091568,...,0.030325,-0.182683,-0.202423,0.645939,0.053046,0.055527,1.318302,0.710148,pre-void,1
4,3.869728,0.654555,0.289030,0.511981,9.715650,0.954451,0.016935,1.335001,5.997071,6.592176,...,0.064735,-0.076608,-0.258715,0.694073,0.105261,0.067007,1.454805,0.810148,pre-void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3885,37.230766,-0.449663,0.154201,0.225975,3.050020,0.983733,0.016543,0.885965,0.472245,8.890633,...,0.058613,0.129760,0.162830,0.912583,0.199245,0.020227,0.728769,0.729865,post-void,41
3886,37.826326,-0.389008,0.166484,0.179044,2.981789,0.977980,0.016543,0.898322,0.472245,4.216030,...,0.095991,0.012482,0.040458,0.893438,0.178505,0.019224,0.716762,0.719302,post-void,41
3887,38.421886,-0.325674,0.148317,0.128061,3.173577,0.969200,0.016543,1.050550,0.472245,3.165288,...,0.095572,-0.221048,-0.263422,0.879072,0.147624,0.011936,0.705305,0.771008,post-void,41
3888,39.017446,-0.289582,0.108687,0.095670,2.996432,0.963300,0.016543,1.398357,0.472245,5.069573,...,0.051392,-0.233781,-0.252719,0.871507,0.283301,0.008837,0.683820,0.809710,post-void,41


## Data 9
classes: pre-void, void, non-void

fs: original

overlap: 0.0

sliding window: 4s

filename: three_class_raw_4s_no.csv

In [23]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data]    
    fs = 1 / df['time'].diff().median()
    analyzer = GenerateFeatures(fs, window_duration=4.0, overlap=0.0)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing signals: 100%|██████████| 5/5 [00:00<00:00, 76.16it/s]


In [24]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/feature_set_4/three_class_raw_4s_no.csv', index=False)
final_features

,center_time,acc_x_mean,acc_x_std,acc_x_energy,acc_x_jerk,acc_x_autocorr_peak,acc_x_autocorr_lag,acc_x_sample_entropy,acc_x_dominant_frequency,acc_x_spectral_centroid,...,gyr_mag_high_band_power,acc_xy_corr,acc_xz_corr,acc_yz_corr,acc_gyr_corr,tilt_var,acc_magnitude_mean,gyr_magnitude_mean,label,experiment_id
0,1.998371,0.895036,0.748686,1.361620,17.055717,0.933704,0.016935,0.721129,5.535758,4.783760,...,0.216851,0.057595,-0.123856,0.594318,0.382591,0.122478,1.382740,1.617914,pre-void,1
1,5.995114,0.611901,0.725789,0.901192,17.138390,0.910789,0.016935,1.166932,2.306566,3.765380,...,0.177183,-0.193035,-0.285993,0.714811,-0.011644,0.449817,1.664570,1.567664,pre-void,1
2,9.991857,0.624439,0.438269,0.582004,13.035493,0.926953,0.016935,1.325225,0.922626,5.394173,...,0.173703,0.009580,0.169398,0.605772,0.258683,0.365241,1.330382,1.598731,pre-void,1
3,13.988600,0.295423,0.460685,0.299506,12.222025,0.860789,0.016935,1.240564,5.997071,5.580798,...,0.163644,0.021715,0.255598,0.625402,0.368199,0.168923,1.133725,1.170979,pre-void,1
4,17.985343,0.394139,1.000649,1.156644,11.038946,0.965989,0.016935,0.531641,0.461313,7.017798,...,0.091636,-0.216172,-0.290915,0.638848,0.478585,0.115481,1.548812,1.440376,void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,22.019174,0.580965,0.140694,0.357315,3.284957,0.985768,0.016543,1.091516,0.472245,6.945493,...,0.098274,0.426761,0.426853,0.897468,-0.142547,0.001265,1.901990,0.790107,void,41
579,26.022660,0.662380,0.124837,0.454331,3.867538,0.985636,0.016543,1.384769,0.472245,6.132933,...,0.060261,0.275066,0.271008,0.948491,0.373640,0.014303,1.105698,0.930813,void,41
580,30.026146,0.501795,0.416167,0.424994,4.242243,0.981126,0.016543,0.483491,0.472245,1.934547,...,0.087974,-0.391642,-0.562127,0.912561,-0.494243,0.091467,0.719994,1.765990,post-void,41
581,34.029632,0.336264,0.569845,0.437797,3.629799,0.988654,0.016543,0.347153,0.472245,2.183934,...,0.085143,-0.187551,-0.418936,0.842823,-0.494132,0.064215,0.696150,1.183135,post-void,41


## Data 9b
classes: pre-void, void, non-void

fs: original

overlap: 0.5

sliding window: 4s

filename: three_class_raw_4s_yo_0.5.csv

In [25]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data]    
    fs = 1 / df['time'].diff().median()
    analyzer = GenerateFeatures(fs, window_duration=4.0, overlap=0.5)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing signals: 100%|██████████| 5/5 [00:00<00:00, 40.81it/s]


In [26]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/feature_set_4/three_class_raw_4s_0.5.csv', index=False)
final_features

,center_time,acc_x_mean,acc_x_std,acc_x_energy,acc_x_jerk,acc_x_autocorr_peak,acc_x_autocorr_lag,acc_x_sample_entropy,acc_x_dominant_frequency,acc_x_spectral_centroid,...,gyr_mag_high_band_power,acc_xy_corr,acc_xz_corr,acc_yz_corr,acc_gyr_corr,tilt_var,acc_magnitude_mean,gyr_magnitude_mean,label,experiment_id
0,1.998371,0.895036,0.748686,1.361620,17.055717,0.933704,0.016935,0.721129,5.535758,4.783760,...,0.216851,0.057595,-0.123856,0.594318,0.382591,0.122478,1.382740,1.617914,pre-void,1
1,3.996743,0.572287,0.434528,0.516326,10.805491,0.915848,0.016935,1.193320,5.997071,7.018773,...,0.073888,-0.220058,-0.415200,0.664340,0.170429,0.091237,1.459578,1.052344,pre-void,1
2,5.995114,0.611901,0.725789,0.901192,17.138390,0.910789,0.016935,1.166932,2.306566,3.765380,...,0.177183,-0.193035,-0.285993,0.714811,-0.011644,0.449817,1.664570,1.567664,pre-void,1
3,7.993486,0.703575,0.690098,0.971253,18.126929,0.892343,0.016935,1.094785,5.997071,5.076948,...,0.237313,-0.082978,0.007834,0.637973,0.196542,0.341263,1.456162,1.780506,pre-void,1
4,9.991857,0.624439,0.438269,0.582004,13.035493,0.926953,0.016935,1.325225,0.922626,5.394173,...,0.173703,0.009580,0.169398,0.605772,0.258683,0.365241,1.330382,1.598731,pre-void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1143,30.026146,0.501795,0.416167,0.424994,4.242243,0.981126,0.016543,0.483491,0.472245,1.934547,...,0.087974,-0.391642,-0.562127,0.912561,-0.494243,0.091467,0.719994,1.765990,post-void,41
1144,32.027889,0.501798,0.412469,0.421932,4.192192,0.982872,0.016543,0.518905,0.472245,1.538961,...,0.090949,0.383206,0.304608,0.943293,-0.523889,0.131342,0.681876,1.699226,post-void,41
1145,34.029632,0.336264,0.569845,0.437797,3.629799,0.988654,0.016543,0.347153,0.472245,2.183934,...,0.085143,-0.187551,-0.418936,0.842823,-0.494132,0.064215,0.696150,1.183135,post-void,41
1146,36.031375,-0.331151,0.355019,0.235700,3.238999,0.979152,0.016543,0.464316,0.472245,1.733627,...,0.054262,-0.494100,-0.590044,0.916520,-0.518783,0.049235,0.659370,1.079353,post-void,41


## Data 9c
classes: pre-void, void, non-void

fs: original

overlap: 0.8

sliding window: 4s

filename: three_class_raw_4s_yo_0.8.csv

In [27]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data]    
    fs = 1 / df['time'].diff().median()
    analyzer = GenerateFeatures(fs, window_duration=4.0, overlap=0.8)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing signals: 100%|██████████| 5/5 [00:00<00:00, 16.54it/s]


In [28]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/feature_set_4/three_class_raw_4s_0.8.csv', index=False)
final_features

,center_time,acc_x_mean,acc_x_std,acc_x_energy,acc_x_jerk,acc_x_autocorr_peak,acc_x_autocorr_lag,acc_x_sample_entropy,acc_x_dominant_frequency,acc_x_spectral_centroid,...,gyr_mag_high_band_power,acc_xy_corr,acc_xz_corr,acc_yz_corr,acc_gyr_corr,tilt_var,acc_magnitude_mean,gyr_magnitude_mean,label,experiment_id
0,1.998371,0.895036,0.748686,1.361620,17.055717,0.933704,0.016935,0.721129,5.535758,4.783760,...,0.216851,0.057595,-0.123856,0.594318,0.382591,0.122478,1.382740,1.617914,pre-void,1
1,2.794333,0.833254,0.676522,1.151995,14.364272,0.932624,0.016935,0.821885,2.306566,4.126660,...,0.042831,0.067296,-0.216927,0.695097,0.313445,0.116908,1.437198,1.259782,pre-void,1
2,3.590294,0.643343,0.329404,0.522398,9.772603,0.956665,0.016935,1.293173,5.535758,6.599218,...,0.063511,-0.220554,-0.345623,0.667577,0.211965,0.076889,1.414341,0.828408,pre-void,1
3,4.386256,0.602633,0.553150,0.669142,11.657440,0.933535,0.016935,0.988422,5.535758,7.018464,...,0.070615,-0.115024,-0.376673,0.683565,0.188017,0.141010,1.542166,1.195111,pre-void,1
4,5.182217,0.569451,0.682941,0.790683,15.973025,0.905800,0.016935,0.958255,0.461313,3.945885,...,0.093801,-0.136942,-0.222975,0.700738,0.096305,0.314774,1.590110,1.388882,pre-void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2850,35.353098,-0.124197,0.543462,0.310776,3.362578,0.982179,0.016543,0.284449,0.472245,1.480385,...,0.072581,-0.478103,-0.662771,0.878946,-0.535084,0.060621,0.657649,1.092732,post-void,41
2851,36.147178,-0.357938,0.315084,0.227398,3.176665,0.983990,0.016543,0.546518,0.472245,2.091256,...,0.052474,-0.479446,-0.565518,0.915432,-0.520991,0.047146,0.660932,1.070773,post-void,41
2852,36.941258,-0.437636,0.147690,0.213337,3.064425,0.987127,0.016543,1.040378,0.472245,7.692625,...,0.056008,0.108805,0.145466,0.913638,0.129272,0.022113,0.694571,0.722838,post-void,41
2853,37.735338,-0.399255,0.164655,0.186515,3.057450,0.981288,0.016543,0.889568,0.472245,4.544649,...,0.083897,0.156929,0.212056,0.904841,0.151337,0.022148,0.711944,0.704397,post-void,41


## Data 10
classes: pre-void, void, non-void

fs: original

overlap: 0.0

sliding window: 5s

filename: three_class_raw_5s_no.csv

In [29]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data]    
    fs = 1 / df['time'].diff().median()
    analyzer = GenerateFeatures(fs, window_duration=5.0, overlap=0.0)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing signals: 100%|██████████| 5/5 [00:00<00:00, 81.10it/s]


In [30]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/feature_set_4/three_class_raw_5s_no.csv', index=False)
final_features

,center_time,acc_x_mean,acc_x_std,acc_x_energy,acc_x_jerk,acc_x_autocorr_peak,acc_x_autocorr_lag,acc_x_sample_entropy,acc_x_dominant_frequency,acc_x_spectral_centroid,...,gyr_mag_high_band_power,acc_xy_corr,acc_xz_corr,acc_yz_corr,acc_gyr_corr,tilt_var,acc_magnitude_mean,gyr_magnitude_mean,label,experiment_id
0,2.497964,0.833686,0.688622,1.169233,15.691631,0.933892,0.016935,0.815896,5.535758,4.853708,...,0.160094,-0.008450,-0.204499,0.642213,0.268746,0.149712,1.490904,1.466923,pre-void,1
1,7.493893,0.597887,0.699962,0.847416,17.382356,0.904367,0.016935,1.128113,5.997071,4.312535,...,0.247240,-0.174339,-0.208124,0.662307,0.160058,0.421337,1.500251,1.820409,pre-void,1
2,12.489822,0.505559,0.478334,0.484393,12.490924,0.913832,0.016935,1.302996,3.229192,5.510294,...,0.183133,0.087968,0.355857,0.609023,0.330021,0.145964,1.197521,1.251777,pre-void,1
3,17.485750,0.319619,0.915191,0.939730,10.836993,0.961483,0.016935,0.570453,0.461313,6.076926,...,0.082872,-0.214670,-0.302200,0.649357,0.462938,0.126187,1.459506,1.377422,void,1
4,22.481679,0.984424,0.390305,1.121429,7.930420,0.980729,0.016935,1.253081,0.461313,5.766811,...,0.069111,-0.268736,-0.292934,0.666481,-0.041202,0.028393,2.041217,0.861220,void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,17.486301,0.495251,0.126030,0.261157,3.735106,0.980471,0.016543,1.242064,0.472245,5.373809,...,0.099316,-0.088897,-0.111420,0.956869,-0.526379,0.011950,1.678247,1.106769,void,41
461,22.482387,0.602377,0.141104,0.382768,3.276323,0.988255,0.016543,1.155217,0.472245,6.919583,...,0.080861,0.438385,0.422745,0.960081,-0.177986,0.003536,1.782526,0.856462,void,41
462,27.478473,0.717196,0.156163,0.538757,3.923895,0.988171,0.016543,1.189028,0.472245,6.099198,...,0.074021,0.502067,0.506366,0.952155,-0.014039,0.031198,1.004702,0.959015,void,41
463,32.474559,0.452289,0.411578,0.373962,4.124711,0.983118,0.016543,0.549684,0.472245,1.734082,...,0.082063,0.235451,0.148637,0.930147,-0.521870,0.153890,0.638233,1.784415,post-void,41


## Data 10b
classes: pre-void, void, non-void

fs: original

overlap: 0.5

sliding window: 5s

filename: three_class_raw_5s_yo_0.5.csv

In [31]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data]    
    fs = 1 / df['time'].diff().median()
    analyzer = GenerateFeatures(fs, window_duration=5.0, overlap=0.5)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing signals: 100%|██████████| 5/5 [00:00<00:00, 44.85it/s]


In [32]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/feature_set_4/three_class_raw_5s_0.5.csv', index=False)
final_features

,center_time,acc_x_mean,acc_x_std,acc_x_energy,acc_x_jerk,acc_x_autocorr_peak,acc_x_autocorr_lag,acc_x_sample_entropy,acc_x_dominant_frequency,acc_x_spectral_centroid,...,gyr_mag_high_band_power,acc_xy_corr,acc_xz_corr,acc_yz_corr,acc_gyr_corr,tilt_var,acc_magnitude_mean,gyr_magnitude_mean,label,experiment_id
0,2.497964,0.833686,0.688622,1.169233,15.691631,0.933892,0.016935,0.815896,5.535758,4.853708,...,0.160094,-0.008450,-0.204499,0.642213,0.268746,0.149712,1.490904,1.466923,pre-void,1
1,5.004396,0.619580,0.656177,0.814448,15.264157,0.916663,0.016935,0.972591,0.461313,3.597922,...,0.111833,-0.154433,-0.231321,0.671232,0.151762,0.298160,1.511610,1.304978,pre-void,1
2,7.510828,0.595686,0.700628,0.845721,17.371547,0.904974,0.016935,1.119555,5.997071,4.321277,...,0.247824,-0.174816,-0.205896,0.659434,0.165140,0.426601,1.494697,1.821169,pre-void,1
3,10.017260,0.634790,0.432010,0.589591,12.569248,0.926678,0.016935,1.416838,0.922626,5.196848,...,0.218247,0.017786,0.159564,0.629451,0.206477,0.354994,1.348802,1.559237,pre-void,1
4,12.523692,0.505274,0.478102,0.483884,12.399092,0.912642,0.016935,1.325424,3.229192,5.554571,...,0.187358,0.088548,0.356868,0.608757,0.330102,0.146376,1.199234,1.251726,pre-void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900,27.478473,0.717196,0.156163,0.538757,3.923895,0.988171,0.016543,1.189028,0.472245,6.099198,...,0.074021,0.502067,0.506366,0.952155,-0.014039,0.031198,1.004702,0.959015,void,41
901,29.976516,0.549039,0.389334,0.453024,3.992223,0.984925,0.016543,0.502883,0.472245,2.518631,...,0.074436,-0.238914,-0.383357,0.931418,-0.529274,0.091513,0.758872,1.607015,post-void,41
902,32.474559,0.452289,0.411578,0.373962,4.124711,0.983118,0.016543,0.549684,0.472245,1.734082,...,0.082063,0.235451,0.148637,0.930147,-0.521870,0.153890,0.638233,1.784415,post-void,41
903,34.972602,0.032065,0.611131,0.374509,3.534878,0.985500,0.016543,0.260790,0.472245,1.704404,...,0.072802,-0.341350,-0.551458,0.878566,-0.551078,0.055087,0.702076,1.001189,post-void,41


## Data 10c
classes: pre-void, void, non-void

fs: original

overlap: 0.8

sliding window: 5s

filename: three_class_raw_5s_yo_0.8.csv

In [33]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data]    
    fs = 1 / df['time'].diff().median()
    analyzer = GenerateFeatures(fs, window_duration=5.0, overlap=0.8)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing signals: 100%|██████████| 5/5 [00:00<00:00, 17.56it/s]


In [34]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/feature_set_4/three_class_raw_5s_0.8.csv', index=False)
final_features

,center_time,acc_x_mean,acc_x_std,acc_x_energy,acc_x_jerk,acc_x_autocorr_peak,acc_x_autocorr_lag,acc_x_sample_entropy,acc_x_dominant_frequency,acc_x_spectral_centroid,...,gyr_mag_high_band_power,acc_xy_corr,acc_xz_corr,acc_yz_corr,acc_gyr_corr,tilt_var,acc_magnitude_mean,gyr_magnitude_mean,label,experiment_id
0,2.497964,0.833686,0.688622,1.169233,15.691631,0.933892,0.016935,0.815896,5.535758,4.853708,...,0.160094,-0.008450,-0.204499,0.642213,0.268746,0.149712,1.490904,1.466923,pre-void,1
1,3.497150,0.680944,0.584940,0.805840,12.807200,0.927355,0.016935,0.992115,5.997071,5.732905,...,0.058255,-0.109024,-0.418378,0.687438,0.194171,0.117704,1.476697,1.306016,pre-void,1
2,4.496336,0.646347,0.549188,0.719371,13.218291,0.932091,0.016935,1.096323,0.461313,4.923995,...,0.072402,-0.203705,-0.353001,0.664274,0.202576,0.227829,1.492238,1.181991,pre-void,1
3,5.495521,0.633710,0.663685,0.842065,15.591740,0.916622,0.016935,1.052211,2.306566,4.119854,...,0.162763,-0.191295,-0.278731,0.707437,0.096704,0.377026,1.548377,1.428480,pre-void,1
4,6.494707,0.607837,0.655408,0.799025,15.600146,0.915122,0.016935,1.054250,5.997071,4.273793,...,0.192523,-0.175506,-0.275325,0.700469,-0.009021,0.406979,1.607442,1.571665,pre-void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2238,34.261238,0.214183,0.608492,0.416137,3.523924,0.990189,0.016543,0.309091,0.472245,1.704196,...,0.080257,-0.313596,-0.515971,0.874112,-0.469103,0.072364,0.696956,1.174356,post-void,41
2239,35.253838,-0.038777,0.594069,0.354422,3.347236,0.988610,0.016543,0.248569,0.472245,1.584962,...,0.073905,-0.397834,-0.579488,0.896929,-0.470004,0.058204,0.705968,1.051261,post-void,41
2240,36.246438,-0.261329,0.392532,0.222374,3.231489,0.984082,0.016543,0.440879,0.472245,1.758453,...,0.060920,-0.500245,-0.627450,0.909248,-0.419773,0.062533,0.650583,1.032746,post-void,41
2241,37.239038,-0.398670,0.158961,0.184207,3.187045,0.983732,0.016543,1.036999,0.472245,5.236208,...,0.070222,0.102914,0.139137,0.914421,-0.200275,0.025160,0.679709,0.786416,post-void,41
